In [1]:
import time
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/mh-patient-drug-switch-prediction/DS_ML_Recruitment_V2.0/Sample Submission.csv
/kaggle/input/mh-patient-drug-switch-prediction/DS_ML_Recruitment_V2.0/test_data.csv
/kaggle/input/mh-patient-drug-switch-prediction/DS_ML_Recruitment_V2.0/train_data.csv
/kaggle/input/mh-patient-drug-switch-prediction/DS_ML_Recruitment_V2.0/train_labels.csv
/kaggle/input/mh-patient-drug-switch-prediction/DS_ML_Recruitment_V2.0/fitness_values_2.csv
/kaggle/input/mh-patient-drug-switch-prediction/DS_ML_Recruitment_V2.0/_DS_Store


In [2]:
train = pd.read_csv('/kaggle/input/mh-patient-drug-switch-prediction/DS_ML_Recruitment_V2.0/train_data.csv')
# test = pd.read_csv('/kaggle/input/mh-patient-drug-switch-prediction/Drug_Switch_Prediction_ParticipantsData/test_data.csv')
ftns = pd.read_csv('/kaggle/input/mh-patient-drug-switch-prediction/DS_ML_Recruitment_V2.0/fitness_values_2.csv')
labels = pd.read_csv('/kaggle/input/mh-patient-drug-switch-prediction/DS_ML_Recruitment_V2.0/train_labels.csv')
# s = pd.read_csv('/kaggle/input/mh-patient-drug-switch-prediction/Drug_Switch_Prediction_ParticipantsData/Sample Submission.csv')
# Any results you write to the current directory are saved as output.

start=time.time()

import pandas as pd
import numpy as np
start

1578958294.417247

In [3]:
train_data = pd.merge(train, labels, on='patient_id', how='left')
del train, labels

In [4]:
time_var = 'event_time'
id_var = 'patient_id'
y_var = 'outcome_flag'
from tqdm import tqdm_notebook as tqdm

In [5]:
def fitness_calculation(data):
    if ((data['sd_0'] == 0 ) and (data['sd_1'] == 0)) and (((data['avg_0'] == 0) and (data['avg_1'] != 0)) or ((data['avg_0'] != 0) and (data['avg_1'] == 0))):
        return 9999999999
    elif (((data['sd_0'] == 0 ) and (data['sd_1'] != 0)) or ((data['sd_0'] != 0) and (data['sd_1'] == 0))) and (data['avg_0'] == data['avg_1']):
        return 1
    elif ((data['sd_0'] != 0 ) and (data['sd_1'] != 0)) and (data['avg_0'] != 0):
        return ((data['avg_1']/data['sd_1'])/(data['avg_0']/data['sd_0']))
    elif ((data['sd_0'] != 0 ) and (data['sd_1'] != 0)) and ((data['avg_0'] == 0) and (data['avg_1'] != 0)):
        return 9999999999
    else:
        return 1

In [6]:
column = ['event_name','specialty','plan_type']

In [7]:
lavg1=[]
lsd1=[]
lavg0=[]
lsd0=[]
fn=[]
feature_name = 'recency__event_name__event_10'

values = train_data['event_name'].unique()


for k in tqdm(column):
    _data=train_data[[k,'patient_id', 'outcome_flag', 'event_time']].groupby(['patient_id',k, 'outcome_flag'])['event_time'].min().reset_index()
    for j in tqdm(train_data[k].unique()):
        patient_level_feature=_data[_data[k]==j].drop([k],axis=1)
        patient_level_feature.columns = ['patient_id', 'outcome_flag', 'feature_value']
        x1=patient_level_feature[(patient_level_feature['outcome_flag']==1) & (patient_level_feature['feature_value']!=9999999999)]['feature_value']
        x0=patient_level_feature[(patient_level_feature['outcome_flag']==0) & (patient_level_feature['feature_value']!=9999999999)]['feature_value']
        lavg1.append(x1.mean())
        lsd1.append(x1.std())
        lavg0.append(x0.mean())
        lsd0.append( x0.std())
        fn.append('recency_'+str(k)+'__'+str(j))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


In [8]:
## Record all the above stats for using the below naming convention.
fitness = pd.DataFrame([fn, lavg1, lavg0, lsd1, lsd0]).transpose().fillna(0)
fitness.columns = ['feature_name', 'avg_1', 'avg_0', 'sd_1', 'sd_0']
fitness['fitness_value'] = fitness.apply(fitness_calculation, axis=1)
fitness ## calculated Fitness Score..

,feature_name,avg_1,avg_0,sd_1,sd_0,fitness_value
0,recency_event_name__event_1,415.562261,402.517278,315.016268,307.536555,1.007895
1,recency_event_name__event_2,334.024691,308.916367,312.396980,298.470307,1.033075
2,recency_event_name__event_3,430.727273,224.280543,277.010141,274.041707,1.899904
3,recency_event_name__event_4,334.842105,326.087912,223.508206,305.235029,1.402317
4,recency_event_name__event_5,478.988636,455.197872,306.338123,315.533077,1.083849
...,...,...,...,...,...,...
750,recency_plan_type__plan_2,18.890511,32.524478,35.987631,57.952816,0.935308
751,recency_plan_type__plan_3,118.685185,155.620808,185.977700,213.227405,0.874402
752,recency_plan_type__plan_4,95.146079,118.374618,152.668850,166.712067,0.877706
753,recency_plan_type__plan_5,10.440454,24.302789,14.278903,75.560492,2.273334


In [9]:


# for k in ['avg_1', 'avg_0', 'sd_1', 'sd_0']:
    

In [10]:
rec=ftns[ftns.feature_name.str.startswith("recency")]
print(rec.shape,fitness.shape)
rec.sort_values('feature_name',inplace=True)
fitness.sort_values('feature_name',inplace=True)

from sklearn.metrics import mean_squared_error
for k in ['avg_1', 'avg_0', 'sd_1', 'sd_0']:
    print(mean_squared_error(rec[k],fitness[k]))

(755, 6) (755, 6)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


6.208903629791646e-16
7.192104382378077e-16
7.200268016887078e-16
8.037776438486295e-16


In [11]:
# train_data[train_data['event_time']<60].groupby([id_var, 'plan_type']).agg({time_var: len}).reset_index().sort_values(['patient_id','plan_type'])

In [12]:
# train_data.groupby([id_var, 'specialty']).agg({'event_time':lambda x : (x<60).count()}).reset_index()

In [13]:
# s=train_data.groupby([id_var, 'plan_type']).agg({'event_time':lambda x : sum(x<60)}).reset_index()
# # s[s['event_time']<30]
# s

In [14]:
# s[s.duplicated()==True]

In [15]:

# np.dot(['event_name'],['1','2'])

In [16]:

# for t in tqdm(([x for x in range(30,0,-30)][::-1])):
    

In [17]:
# ft

In [18]:
ftns[ftns.feature_name.str.startswith('frequency_30_plan_type')]

,feature_name,avg_0,avg_1,sd_0,sd_1,fitness_value
27719,frequency_30_plan_type__plan_1,20.434532,6.966898,95.402888,27.011553,1.204167
27720,frequency_30_plan_type__plan_2,10.275428,14.025746,28.899216,22.660379,1.740784
27721,frequency_30_plan_type__plan_3,0.914091,0.230486,8.038264,1.271102,1.594547
27722,frequency_30_plan_type__plan_4,1.236443,1.637107,4.314361,3.849889,1.483786
27723,frequency_30_plan_type__plan_5,1.691065,1.282795,3.581999,3.207992,0.847012
27724,frequency_30_plan_type__plan_6,2.252810,3.139763,3.283995,3.706980,1.234681


In [19]:
from itertools import product
# list(product(train_data['plan_type'].unique().tolist(),train_data[id_var].unique().tolist()))

In [20]:
# pd.DataFrame(list(product(train_data['plan_type'].unique().tolist(),train_data[id_var].unique().tolist())),columns=['s1','p1'])

In [21]:
# _df1 = pd.DataFrame(_freq['feature_name'].unique().tolist(), columns=['feature_name'])
# _df2 = pd.DataFrame(train_data[id_var].unique().tolist(), columns=[id_var])
# _df1['key'] = 1
# _df2['key'] = 1
# _freqTotal = pd.merge(_df2, _df1, on='key')

In [22]:
# list(product(train_data['specialty'].unique(),[x for x in range(1080,0,-30)]))

In [23]:
# event_time_list=list(product(column,[x for x in range(1080,0,-30)]))
# _df3=train_data[[id_var,y_var]].drop_duplicates().reset_index(drop=True)
# ft=pd.DataFrame()
# ft1=pd.DataFrame()
# column=['plan_type']
# for kk in tqdm(column):
#     for t in tqdm(([x for x in range(1080,0,-30)][::-1])):
        
# for kk,t in tqdm(event_time_list):
#     _data = train_data[(train_data[time_var]<=int(t))].reset_index(drop=True)
#     _freq = _data[[id_var, kk, time_var]].groupby([id_var, kk]).agg({time_var: len}).reset_index()
#     _freq.columns = [id_var, 'feature_name', 'feature_value']
#     _freq['feature_name'] = 'frequency_' + str(t) + '_' +str(kk)+'__'+ _freq['feature_name'].astype(str)
#     _freq = _freq.reset_index(drop=True)
#     freqTotal=pd.DataFrame(product(train_data[id_var].unique(),_freq['feature_name'].unique()),columns=['patient_id','feature_name'])
#     freqTotal = pd.merge(freqTotal, _freq, on=[id_var, 'feature_name'], how='left')
#     freqTotal.fillna(0, inplace=True)
#     freqTotal = freqTotal.merge(_df3, on=id_var, how='left')
#     xx1=freqTotal.loc[freqTotal[y_var]==1,['feature_name', 'feature_value']].groupby('feature_name').agg({'feature_value':['mean','std']}).reset_index()
#     xx1.columns=['feature_name','avg_1','sd_1']
#     xx0=freqTotal.loc[freqTotal[y_var]==0,['feature_name', 'feature_value']].groupby('feature_name').agg({'feature_value':['mean','std']}).reset_index()
#     xx0.columns=['feature_name','avg_0','sd_0']
#     _fitness_value = pd.merge(xx1, xx0, on='feature_name', how='left')
#     ft=ft.append(_fitness_value,ignore_index=True)

_df3=train_data[[id_var,y_var]].drop_duplicates().reset_index(drop=True)
ft=pd.DataFrame()
ft1=pd.DataFrame()
for kk in tqdm(column):
    for t in tqdm(([x for x in range(1080,0,-30)][::-1])):
#         _data = train_data[(train_data[time_var]<=int(t))].reset_index(drop=True)
        _freq = train_data[(train_data[time_var]<=int(t))][[id_var, kk, time_var]].groupby([id_var, kk]).agg({time_var: len}).reset_index()
        _freq.columns = [id_var, 'feature_name', 'feature_value']
        _freq['feature_name'] = 'frequency_' + str(t) + '_' +str(kk)+'__'+ _freq['feature_name'].astype(str)
        _freq = _freq.reset_index(drop=True)
        freqTotal=pd.DataFrame(product(train_data[id_var].unique(),_freq['feature_name'].unique()),columns=['patient_id','feature_name'])
        freqTotal = pd.merge(freqTotal, _freq, on=[id_var, 'feature_name'], how='left')
        freqTotal.fillna(0, inplace=True)
        freqTotal = freqTotal.merge(_df3, on=id_var, how='left')
        xx1=freqTotal.loc[freqTotal[y_var]==1,['feature_name', 'feature_value']].groupby('feature_name').agg({'feature_value':['mean','std']}).reset_index()
        xx1.columns=['feature_name','avg_1','sd_1']
        xx0=freqTotal.loc[freqTotal[y_var]==0,['feature_name', 'feature_value']].groupby('feature_name').agg({'feature_value':['mean','std']}).reset_index()
        xx0.columns=['feature_name','avg_0','sd_0']
        _fitness_value = pd.merge(xx1, xx0, on='feature_name', how='left')
        ft=ft.append(_fitness_value,ignore_index=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [24]:
all_specs=[]
for k,j in list(product(train_data['specialty'].unique(),[x for x in range(1080,0,-30)])):
    if 'frequency_' + str(j) + '_specialty' +'__'+ k not in ft.feature_name.values:
        ft=ft.append({'feature_name':'frequency_' + str(j) + '_specialty' +'__'+ k,
                   'avg_1':0,'avg_0':0,'sd_0':0,'sd_1':0},ignore_index=True)
                 
#         all_specs.append('frequency_' + str(j) + '_specialty' +'__'+ k)
# len(all_specs)
ft

# ft['feature_name'].append(np.setdiff1d(all_specs,ft.feature_name.values))

,feature_name,avg_1,sd_1,avg_0,sd_0
0,frequency_30_event_name__event_1,0.154475,1.186530,0.170694,1.527714
1,frequency_30_event_name__event_10,0.038414,0.520389,0.026271,0.398291
2,frequency_30_event_name__event_100,0.003678,0.078224,0.005128,0.301341
3,frequency_30_event_name__event_101,0.299550,1.580160,0.322141,2.249063
4,frequency_30_event_name__event_102,0.033510,0.572343,0.065468,1.082510
...,...,...,...,...,...
27175,frequency_150_specialty__spec_233,0.000000,0.000000,0.000000,0.000000
27176,frequency_120_specialty__spec_233,0.000000,0.000000,0.000000,0.000000
27177,frequency_90_specialty__spec_233,0.000000,0.000000,0.000000,0.000000
27178,frequency_60_specialty__spec_233,0.000000,0.000000,0.000000,0.000000


In [25]:
ft

,feature_name,avg_1,sd_1,avg_0,sd_0
0,frequency_30_event_name__event_1,0.154475,1.186530,0.170694,1.527714
1,frequency_30_event_name__event_10,0.038414,0.520389,0.026271,0.398291
2,frequency_30_event_name__event_100,0.003678,0.078224,0.005128,0.301341
3,frequency_30_event_name__event_101,0.299550,1.580160,0.322141,2.249063
4,frequency_30_event_name__event_102,0.033510,0.572343,0.065468,1.082510
...,...,...,...,...,...
27175,frequency_150_specialty__spec_233,0.000000,0.000000,0.000000,0.000000
27176,frequency_120_specialty__spec_233,0.000000,0.000000,0.000000,0.000000
27177,frequency_90_specialty__spec_233,0.000000,0.000000,0.000000,0.000000
27178,frequency_60_specialty__spec_233,0.000000,0.000000,0.000000,0.000000


In [26]:
ftns[(ftns.feature_name.str.contains("specialty")) &(ftns.feature_name.str.contains("frequency"))]

,feature_name,avg_0,avg_1,sd_0,sd_1,fitness_value
19475,frequency_30_specialty__spec_1,30.709961,20.992644,97.554595,32.366727,2.060330
19476,frequency_30_specialty__spec_2,2.150534,1.867593,8.998155,3.960098,1.973256
19477,frequency_30_specialty__spec_3,0.716704,0.688190,1.865488,1.722238,1.040082
19478,frequency_30_specialty__spec_5,0.350028,0.380875,1.394223,1.343806,1.128950
19479,frequency_30_specialty__spec_8,0.083521,0.093175,0.491565,0.505390,1.085079
...,...,...,...,...,...,...
27714,frequency_1080_specialty__spec_239,0.000140,0.000000,0.016762,0.000000,1.000000
27715,frequency_1080_specialty__spec_241,0.000070,0.000000,0.008381,0.000000,1.000000
27716,frequency_1080_specialty__spec_242,0.000000,0.000409,0.000000,0.020215,1.000000
27717,frequency_1080_specialty__spec_100,0.000070,0.000000,0.008381,0.000000,1.000000


In [27]:
frq=ftns[ftns.feature_name.str.startswith("frequency")]
print(frq.shape,ft.shape)
frq.sort_values('feature_name',inplace=True)
ft.sort_values('feature_name',inplace=True)

from sklearn.metrics import mean_squared_error
for k in ['avg_1', 'avg_0', 'sd_1', 'sd_0']:
    print(mean_squared_error(frq[k],ft[k]))

(27180, 6) (27180, 5)
2.571392079616882e-18
3.8009021399129394e-17
3.4259053716776394e-18
4.311963364559292e-18


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
# ft=pd.DataFrame()
# ft1=pd.DataFrame()
# column=['plan_type']
# for kk in tqdm(column):
#     for t in tqdm(([x for x in range(540,0,-30)][::-1])):

#         _data = train_data[(train_data[time_var]<=int(t))].reset_index(drop=True)
# #         print(_data.nunique())
#         _freq = _data[[id_var, kk, time_var]].groupby([id_var, kk]).agg({time_var: len}).reset_index()
#         _freq.columns = [id_var, 'feature_name', 'feature_value']
#         _freq['feature_name'] = 'frequency__' + kk + '__' + _freq['feature_name'].astype(str) + '__' + str(t)
#         _freq = _freq.reset_index(drop=True)
# #         print(_freq.nunique())
#         _df1 = pd.DataFrame(_freq['feature_name'].unique().tolist(), columns=['feature_name'])
#         _df2 = pd.DataFrame(_freq[id_var].unique().tolist(), columns=[id_var])
#         _df1['key'] = 1
#         _df2['key'] = 1
#         _freqTotal = pd.merge(_df2, _df1, on='key')
#         _freqTotal.drop(['key'], axis=1, inplace=True)
#         # _freqTotal=pd.DataFrame({id_var:_freq[id_var].unique().tolist(),'feature_name':_freq['feature_name'].unique().tolist()})
# #         _freqTotal = pd.merge(pd.DataFrame({id_var:_freq[id_var].unique().tolist(),'feature_name':_freq['feature_name'].unique().tolist()}), _freq, on=[id_var, 'feature_name'], how='left')
#         _freqTotal.fillna(0, inplace=True)
#         _df3 = train_data[[id_var,y_var]].drop_duplicates().reset_index(drop=True)
#         _freqTotal = _freqTotal.merge(_df3, on=id_var, how='left')
#         freqTotal = _freqTotal.copy()
#         x1=freqTotal.loc[freqTotal[y_var]==1,['feature_name', 'feature_value']].groupby('feature_name')
#         x0=freqTotal.loc[freqTotal[y_var]==0,['feature_name', 'feature_value']].groupby('feature_name')
#         _avg1 = x1.mean().reset_index()
#         _avg1.columns = ['feature_name', 'avg_1']
#         _sd1 = x1.agg(np.std).reset_index()
#         _sd1.columns = ['feature_name', 'sd_1']
#         _avg0 = x0.mean().reset_index()
#         _avg0.columns = ['feature_name', 'avg_0']
#         _sd0 = x0.agg(np.std).reset_index()
#         _sd0.columns = ['feature_name', 'sd_0']

#         _fitness_value = pd.merge(_avg1, _avg0, on='feature_name', how='left')
#         _fitness_value = pd.merge(_fitness_value, _sd1, on='feature_name', how='left')
#         _fitness_value = pd.merge(_fitness_value, _sd0, on='feature_name', how='left')
# #         print(_fitness_value.shape)
#         ft=ft.append(_fitness_value,ignore_index=True)

In [29]:
# ft=ft.append(_fitness_value)
# ft
# ft[ft.feature_name.isin(ftns.feature_name)]
# ftns[(ftns.feature_name.str.startswith('frequency__specialty')) & (ftns.feature_name.str.endswith('_30'))]
ftns[ftns.feature_name.str.startswith('frequency_')][~ftns[ftns.feature_name.str.startswith('frequency_')].feature_name.isin(ft.feature_name)]

,feature_name,avg_0,avg_1,sd_0,sd_1,fitness_value


In [30]:
ft.shape

(27180, 5)

In [31]:
ft['fitness_value'] = ft.apply(fitness_calculation, axis=1)
# ft

In [32]:
# ft=ft.append(ftns[ftns.feature_name.str.startswith('frequency_')][~ftns[ftns.feature_name.str.startswith('frequency_')].feature_name.isin(ft.feature_name)])

In [33]:
ft.shape

(27180, 6)

In [34]:
ft[ft.feature_name=='frequency__plan_type__plan_6__30']

,feature_name,avg_1,sd_1,avg_0,sd_0,fitness_value


In [35]:
# ftns[755:]
ftns[ftns.feature_name=='frequency__plan_type__plan_6__30']

,feature_name,avg_0,avg_1,sd_0,sd_1,fitness_value


In [36]:
ftns[ftns.feature_name.str.startswith('normChange__specialty')]

,feature_name,avg_0,avg_1,sd_0,sd_1,fitness_value


In [37]:
column

['event_name', 'specialty', 'plan_type']

In [38]:
# event = feature_name.split('__')[1]
# value = feature_name.split('__')[2]
# time = feature_name.split('__')[3]
# __df3 = train_data[[id_var,y_var]].drop_duplicates().reset_index(drop=True)
ftnorm=pd.DataFrame()
# column=['plan_type']
for kk in tqdm(column):
    for t in tqdm(([x for x in range(540,0,-30)][::-1])):
        _data_post = train_data[train_data[time_var]<=int(t)].reset_index(drop=True)
        _data_pre = train_data[train_data[time_var]>int(t)].reset_index(drop=True)
        _freq_post = _data_post[[id_var, kk, time_var]].groupby([id_var, kk]).agg({time_var: len}).reset_index()
        _freq_pre = _data_pre[[id_var, kk, time_var]].groupby([id_var, kk]).agg({time_var: len}).reset_index()
#         print(_freq_post.nunique(),_freq_pre.nunique())
        _freq_post.columns = [id_var, 'feature_name', 'feature_value_post']
        _freq_pre.columns = [id_var, 'feature_name', 'feature_value_pre']
        _freq_post['feature_value_post'] = _freq_post['feature_value_post']/int(t)
        _freq_pre['feature_value_pre'] = _freq_pre['feature_value_pre']/((train_data[time_var].max()) - int(t))
        _normChange = pd.merge(_freq_post, _freq_pre, on=[id_var, 'feature_name'], how='outer')
        _normChange.fillna(0, inplace=True)
        _normChange['feature_value'] = np.where(_normChange['feature_value_post']>_normChange['feature_value_pre'], 1, 0)
        _normChange.drop(['feature_value_post', 'feature_value_pre'], axis=1, inplace=True)
#         _normChange['feature_name'] = 'normChange__' + kk + '__' + _normChange['feature_name'].astype(str) + '__' + str(t)
        _normChange['feature_name'] = 'normChange_' + str(t)+'_'+ kk + '__' + _normChange['feature_name'].astype(str)

        _normChange = _normChange.reset_index(drop=True)
        _df1 = pd.DataFrame(_normChange['feature_name'].unique().tolist(), columns=['feature_name'])
        _df2 = pd.DataFrame(_normChange[id_var].unique().tolist(), columns=[id_var])
        _df1['key'] = 1
        _df2['key'] = 1
        _normTotal = pd.merge(_df2, _df1, on='key')
        _normTotal.drop(['key'], axis=1, inplace=True)
        _normTotal = pd.merge(_normTotal, _normChange, on=[id_var, 'feature_name'], how='left')
        _normTotal.fillna(0, inplace=True)
        
        _normTotal = _normTotal.merge(_df3, on=id_var, how='left')
        normTotal = _normTotal.copy()

#         _avg1 = normTotal.loc[normTotal[y_var]==1,['feature_name', 'feature_value']].groupby('feature_name').mean().reset_index()
#         _avg1.columns = ['feature_name', 'avg_1']
#         _sd1 = normTotal.loc[normTotal[y_var]==1,['feature_name', 'feature_value']].groupby('feature_name').agg(np.std).reset_index()
#         _sd1.columns = ['feature_name', 'sd_1']
#         _avg0 = normTotal.loc[normTotal[y_var]==0,['feature_name', 'feature_value']].groupby('feature_name').mean().reset_index()
#         _avg0.columns = ['feature_name', 'avg_0']
#         _sd0 = normTotal.loc[normTotal[y_var]==0,['feature_name', 'feature_value']].groupby('feature_name').agg(np.std).reset_index()
#         _sd0.columns = ['feature_name', 'sd_0']
        
        xx1=normTotal.loc[normTotal[y_var]==1,['feature_name', 'feature_value']].groupby('feature_name').agg({'feature_value':['mean','std']}).reset_index()
        xx1.columns=['feature_name','avg_1','sd_1']
        xx0=normTotal.loc[normTotal[y_var]==0,['feature_name', 'feature_value']].groupby('feature_name').agg({'feature_value':['mean','std']}).reset_index()
        xx0.columns=['feature_name','avg_0','sd_0']
        _fitness_value = pd.merge(xx1, xx0, on='feature_name', how='left')

#         _fitness_value = pd.merge(_avg1, _avg0, on='feature_name', how='left')
#         _fitness_value = pd.merge(_fitness_value, _sd1, on='feature_name', how='left')
#         _fitness_value = pd.merge(_fitness_value, _sd0, on='feature_name', how='left')
        ftnorm=ftnorm.append(_fitness_value)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [39]:
ftns[(ftns.feature_name.str.startswith('normChange__specialty')) & (ftns.feature_name.str.endswith('_30'))]

,feature_name,avg_0,avg_1,sd_0,sd_1,fitness_value


In [40]:
# fitness = _fitness_value[_fitness_value.feature_name==feature_name]
ftnorm['fitness_value'] = ftnorm.apply(fitness_calculation, axis=1)
ftnorm ## Calculated Fitness Scores for Frequency..

,feature_name,avg_1,sd_1,avg_0,sd_0,fitness_value
0,normChange_30_event_name__event_1,0.035554,0.185213,0.031329,0.174212,1.067443
1,normChange_30_event_name__event_10,0.010625,0.102551,0.009975,0.099378,1.032259
2,normChange_30_event_name__event_100,0.002861,0.053419,0.001405,0.037457,1.427757
3,normChange_30_event_name__event_101,0.055987,0.229943,0.050084,0.218127,1.060407
4,normChange_30_event_name__event_102,0.008173,0.090054,0.007938,0.088742,1.014684
...,...,...,...,...,...,...
1,normChange_540_plan_type__plan_2,0.507152,0.500051,0.379250,0.485217,1.297581
2,normChange_540_plan_type__plan_3,0.118921,0.323762,0.173223,0.378453,0.802491
3,normChange_540_plan_type__plan_4,0.449121,0.497506,0.326426,0.468922,1.296823
4,normChange_540_plan_type__plan_5,0.122599,0.328044,0.166971,0.372963,0.834796


In [41]:
ftnorm[ftnorm.feature_name=='normChange__event_name__event_10__30']

,feature_name,avg_1,sd_1,avg_0,sd_0,fitness_value


In [42]:
ftns[ftns.feature_name=='normChange__event_name__event_10__30']

,feature_name,avg_0,avg_1,sd_0,sd_1,fitness_value


In [43]:
ft.shape,fitness.shape,ftnorm.shape

((27180, 6), (755, 6), (13590, 6))

In [44]:
frq=ftns[ftns.feature_name.str.startswith("norm")]
print(frq.shape,ftnorm.shape)
ftnorm.sort_values('feature_name',inplace=True)
frq.sort_values('feature_name',inplace=True)

from sklearn.metrics import mean_squared_error
for k in ['avg_1', 'avg_0', 'sd_1', 'sd_0']:
    print(mean_squared_error(frq[k],ftnorm[k]))

(13590, 6) (13590, 6)
7.099407226715808e-20
1.0577169004649758e-16
7.827494681553542e-20
7.646027154663013e-20


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [45]:
ft=ft.append(fitness,ignore_index=True)
ft=ft.append(ftnorm,ignore_index=True)
ft.shape

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


(41525, 6)

In [46]:
ft

,avg_0,avg_1,feature_name,fitness_value,sd_0,sd_1
0,4.822563,4.890069,frequency_1020_event_name__event_1,1.205549,15.852724,13.333870
1,1.139014,1.230078,frequency_1020_event_name__event_10,0.909126,5.216169,6.196281
2,0.105296,0.057622,frequency_1020_event_name__event_100,1.697902,1.259210,0.405842
3,7.239393,6.875766,frequency_1020_event_name__event_101,1.259058,17.665725,13.326153
4,1.013557,0.610544,frequency_1020_event_name__event_102,0.998176,6.149789,3.711259
...,...,...,...,...,...,...
41520,0.000351,0.000000,normChange_90_specialty__spec_95,1.000000,0.018738,0.000000
41521,0.000211,0.000000,normChange_90_specialty__spec_96,1.000000,0.014516,0.000000
41522,0.000351,0.000000,normChange_90_specialty__spec_97,1.000000,0.018738,0.000000
41523,0.000000,0.000817,normChange_90_specialty__spec_98,1.000000,0.000000,0.028583


In [47]:
ftns[ftns.feature_name=='normChange__specialty__spec_96__30']

,feature_name,avg_0,avg_1,sd_0,sd_1,fitness_value


In [48]:
ft.to_csv('final_fitness_v2.csv',index=False)

In [49]:
print(time.time()-start)

3021.643055677414
